In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# STEP 1 : CREATE THE CLASS NODE FOR BUILDING THE TREE

In [2]:
class Node:
    def __init__(self, predicted_class):
        self.predicted_class = predicted_class
        self.feature = None
        self.threshold = None
        self.left = None
        self.right = None 

# STEP 2 :  BEST SPLIT FUNCTION

In [3]:
#the goal of this function is to return best_feature and best_threshold 

def best_split(X, y , n_classes, n_features):
    m = len(y)
    if m <= 1:
        return None , None
    #we need to find the gini for the parent node 
    num_parent = [np.sum(y==c) for c in range(n_classes)]
    best_gini = 1.0 - sum([(n/m)**2 for n in num_parent])
    best_threshold , best_feature = None , None 

    #we need to find the best_feature and the best threshold by looping through all the feeatures
    #but we need to get the dataset in the best shape possible for this splitting 
    
    for feature in range(n_features):
        feature_values = X[:, feature]
        threshold, classes = zip(*sorted(zip(feature_values, y))) #first time I'm learning about the *
        
        num_left = [0] * n_classes 
        num_right = num_parent.copy()
        
        for i in range(1, m):
            c = classes[i-1]
            num_left[c] += 1
            num_right[c] -= 1
            
            #find the gini impurity for each partition and the weighted gini too 
            gini_left = 1 - sum([ (num_left[c]/i)**2 for c in range(n_classes)])
            gini_right = 1 - sum([ (num_left[c]/(m-i))**2 for c in range(n_classes)])
            gini = (((i * gini_left) + ((m-i)*gini_right)))/m
            
            if threshold[i-1] == threshold[i]:
                continue
                
            if gini < best_gini:
                best_gini = gini
                best_feature = feature
                best_threshold = (threshold[i] + threshold[i-1]) / 2
    return best_threshold , best_feature
                
                
                
            

            
            

            
        

# STEP 3 : Grow Tree using DFS

In [5]:
def grow_tree(X, y, depth=0 , max_depth= None, n_classes= None):
    num_samples_per_class = [np.sum(y==i) for i in range(n_classes)]
    predicted_class = np.argmax(num_samples_per_class)
    node = Node(predicted_class = predicted_class)
    
    #just like in every depth first search and we need a base case 
    
    if max_depth is not None and depth >= max_depth:
        return node 
        
    n_features = X.shape[1]
    threshold , feature  = best_split(X, y, n_classes, n_features)

    #another case we have to consider is if if feature return none
    if feature is None :
        return node
    #splitting the dataset base on the best feature and best threshold 
    
    indices_left = X[:, feature] < threshold
    X_left, y_left = X[indices_left], y[indices_left]
    X_right , y_right = X[~indices_left], y[~indices_left]

    node.feature = feature
    node.threshold = threshold
    node.left = grow_tree(X_left, y_left, depth+1,max_depth , n_classes)
    node.right =  grow_tree(X_right, y_right, depth+1,max_depth, n_classes)

    return node
    



# Decision tree class

In [6]:
class DecisionTreeClassifier:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.root = None
        self.n_classes = None

    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        self.root = grow_tree(X, y, max_depth=self.max_depth, n_classes=self.n_classes)
        return self

    def predict(self, X):
        return np.array([self._predict_one(x, self.root) for x in X])

    def _predict_one(self, x, node):
        if node.left is None and node.right is None:
            return node.predicted_class
        if x[node.feature_index] < node.threshold:
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

# test the code

In [8]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [11]:
#load the dataset

iris = load_iris()
X= iris.data
y= iris.target

In [12]:
#split the dataset into train and test
X_train ,X_test , y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


In [21]:
tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train, y_train)


In [22]:
y_preds = tree.predict(X_test)


In [23]:
accuracy = accuracy_score(y_test, y_preds)
print(f'Accuracy : {accuracy}')

Accuracy : 0.3
